In [ ]:
import os
import cv2
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import partial

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

from utils.dataset import VideoDataset, MyConcatDataset
from utils.models import TrackNetV2MSE, TrackNetV2NLL
from utils.training import train_model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

%load_ext autoreload
%autoreload 2

# Dataset optimizations

In [ ]:
sequence_length = 3
one_output_frame = True

image_size = (360, 640)

dataset_params = dict(image_size=image_size,
                      sigma=5,
                      sequence_length=sequence_length,
                      heatmap_mode='image',
                      one_output_frame=one_output_frame,
                      drop_duplicate_frames=True,
                      preload_in_memory=True)

# add transforms
dataset_params['transform'] = ToTensor()
dataset_params['target_transform'] = ToTensor()

dataset = VideoDataset(root="../datasets/prova/", duplicate_equality_threshold=1, **dataset_params)


In [ ]:
frames, labels = dataset[29]

In [ ]:

N = 20

st = time.time()
for i in range(N):
    dataset[i]
    print(f"{i+1} of {N}", end = '\r')
print(time.time() - st)

In [ ]:
plt.imshow(frames[:3].numpy().transpose(1, 2, 0))

In [ ]:
def do_raise():
    raise ValueError()

try:
    do_raise()
    print('a')
except ValueError as e:
    print(e)

In [ ]:
cap = cv2.VideoCapture("../datasets/prova/video.MP4")
print(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

st = time.time()
cap.set(cv2.CAP_PROP_POS_FRAMES, 10)
print(time.time() - st)

cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

st = time.time()
cap.set(cv2.CAP_PROP_POS_FRAMES, 100)
print(time.time() - st)

cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

st = time.time()
cap.set(cv2.CAP_PROP_POS_FRAMES, 1000)
print(time.time() - st)

cap.release()


In [ ]:
N = 20

cap = cv2.VideoCapture("../datasets/prova/video.MP4")
fc = cap.get(cv2.CAP_PROP_FRAME_COUNT)

st = time.time()
for i in range(N):
    i = np.random.randint(0, fc)

    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = cap.read()
    if not ret:
        print("AAA")
cap.release()

t1 = time.time() - st
print(t1)


cap = cv2.VideoCapture("../datasets/prova/video.MP4")

st = time.time()
ret = True
i=0
while ret and i<N:
    ret, frame = cap.read()
    if ret:
        i+=1
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        print(f"Frame {i}", end = '\r')
    else:
        print(f"Frame {i} (finished)")
cap.release()
t2 = time.time() - st
print(t2)
print(i)